## Import modules

In [1]:
# def load_data(loc:str, index:list):
#     df = pandas.read_csv(f'data/{loc}_solar19.csv')
#     df['idx'] = index
#     df = df.set_index('idx')
#     return df
# idx = [(i,j,k) for i,j,k in product(range(1), range(365), range(24))]
# weather_ny = load_data('ny', idx)[['DNI', 'Wind Speed']]
# weather_ny = weather_ny.rename(columns= {'DNI': 'dni', 'Wind Speed': 'wind_speed'})
# weather_sd = load_data('sd', idx)[['dni', 'wind_speed']]
# weather_ho = load_data('ho', idx)[['dni', 'wind_speed']]""
# demand_sd = pandas.read_excel('data/HistoricalEMSHourlyLoad-2019.xlsx', index_col= 0)[['SDGE']]
# demand_ho = pandas.read_excel('data/Native_Load_2019.xlsx')[['COAST']]
# demand_ny = pandas.DataFrame(pandas.read_csv('data/NYC_load.csv')['Load']) #from Will and Doga
# ng_price = make_henry_price_df(
#     file_name='data/Henry_Hub_Natural_Gas_Spot_Price_Daily.csv', year=2020)
# ng_price = ng_price.drop(columns= 'scales')
# ng_price = remove_outliers(ng_price, sd_cuttoff = 3)
# ng_price = pandas.concat([ng_price])

In [2]:
import sys

sys.path.append('src')

In [3]:
from pandas import DataFrame
from energiapy.components import *
from energiapy.model.scenario import Scenario

In [4]:
# from energiapy.tasks.task_master import TaskMaster
# t = TaskMaster()

In [5]:
s = Scenario()

In [6]:
# s.h = Horizon({'days': 365, 'hours': 24})
s.h = Horizon(discretizations=[2, 12])

In [7]:
s.n = Network(['madgaon', 'ponje', 'cacoda'])
# s.n = Network(3)

In [8]:
s.nodes

[cacoda, madgaon, ponje]

In [9]:
s.alink = Linkage(source=s.madgaon, sink=s.ponje, bi=True, distance=50)
s.blink = Linkage(source=s.ponje, sink=s.cacoda, bi=False, distance=80)
s.clink = Linkage(source=s.cacoda, sink=s.madgaon, bi=True, distance=100)
s.dlink = Linkage(source=s.madgaon, sink=s.ponje, bi=False, distance=200)

In [10]:
# s.lnd = Land(use={s.madgaon: 100, s.ponje: 200,
#              s.cacoda: 300, s.n: {s.t2: 600}})

In [11]:
a = DataFrame({'a': list(range(2))})
b = DataFrame({'b': list(range(24))})

In [12]:
s.csh = Cash(
    spend={
        s.madgaon: b,
        s.ponje: {s.t1: (2, b)},
        s.cacoda: 300,
        s.n: {s.t0: [a, 60]},
    }
)

# s.csh = Cash(spend=a)

c:\Users\cacodcar\energiapy\src\energiapy\components\_base\_consistent.py:124: UserWarning: csh:Inconsistent temporal scale for ponje at t1. Updating to t2
  warn(
c:\Users\cacodcar\energiapy\src\energiapy\components\_base\_consistent.py:124: UserWarning: csh:Inconsistent temporal scale for n at t0. Updating to t1
  warn(


In [13]:
s.land.use

{n: {t0: None}}

In [14]:
s.lnd = Land(use={s.t2: 100, s.ponje: {s.t0: b},
             s.cacoda: a},
             use_cost = 20)
# s.lnd = Land(use= 2)

c:\Users\cacodcar\energiapy\src\energiapy\components\_base\_consistent.py:88: UserWarning: lnd:Inconsistent temporal scale for ponje at t0. Updating to t2
  warn(


In [25]:
s.lnd.use 

{n: {t2: 100},
 ponje: {t2:      b
  0    0
  1    1
  2    2
  3    3
  4    4
  5    5
  6    6
  7    7
  8    8
  9    9
  10  10
  11  11
  12  12
  13  13
  14  14
  15  15
  16  16
  17  17
  18  18
  19  19
  20  20
  21  21
  22  22
  23  23},
 cacoda: {t1:    a
  0  0
  1  1}}

In [16]:
s.land.inputs()

['use', 'use_cost']

In [17]:
value = s.csh.spend
# value = s.lnd.use

In [18]:
horizon = s.h
network = s.n

In [19]:
s.h.is_multiscale()

True

In [20]:
s.linkages

[alink, alink_, blink, clink, clink_, dlink]

In [21]:
s.madgaon.is_source()

True

In [22]:
s.sources

[cacoda, madgaon, ponje]

In [23]:
s.linkages

[alink, alink_, blink, clink, clink_, dlink]

In [ ]:
s.processes

In [ ]:
s.madgaon.find_links(s.ponje)

In [ ]:
s.madgaon.is_connected(s.ponje)

In [ ]:
a = pandas.DataFrame({'a': list(range(8760))})
b = pandas.DataFrame({'b': list(range(8760))})
cc = DataSet(data=a, horizon=s.h, scaling_max=True)

In [ ]:
s.H2 = Resource(
    discharge={s.t0: [0, 4], s.t1: [22, 34], s.t2: (a, b)}, sell_cost=(0, 10)
)
s.Solar = Resource(gwp=50, eutt=60, consume=BigM,
                   basis='MW', label='Solar Power')
s.Wind = Resource(consume=cc, basis='MW', label='Wind Power')
s.Power = Resource(basis='MW', label='Power generated')
s.Uranium = Resource(
    consume=pandas.DataFrame({'a': [i for i in range(365)]}),
    purchase_cost=42.70 / (250 / 2),
    basis='kg',
    label='Uranium',
)
s.H2_L = Resource(discharge=(0, 23), basis='tons', label='Hydrogen')
s.CO2_AQoff = Resource(basis='tons', label='Carbon dioxide - sequestered')
s.H2O = Resource(consume=(20, 50), purchase_cost=cc,
                 basis='tons', label='Water')
s.CH4 = Resource(consume=[20, 40], purchase_cost=20,
                 basis='tons', label='Natural gas')
s.CO2 = Resource(basis='tons', label='Carbon dioxide', block='Resource')
s.CO2_Vent = Resource(
    basis='tons', label='Carbon dioxide - Vented', sell_cost=(2, 20))
s.O2 = Resource(discharge=[20, BigM], basis='tons', label='Oxygen')
s.CO2_DAC = Resource(basis='tons', label='Carbon dioxide - captured')

In [ ]:
from pydantic import BaseModel

In [ ]:
s.Power.__dict__

In [ ]:
s.Solar.ctypes

## Declare processes

In [ ]:
# s.LiI = Process(store= 2000, conversion = {s.Power: 0.9}, store_loss = 0.2, store_cost= 0.4, capex = 3516428, fopex= 87910, vopex = 0, capacity= [1000,BigM], label='Lithium-ion battery', basis = 'MW')
s.WF = Process(
    gwp=50,
    odp=2900,
    produce=weather_ho[['wind_speed']],
    conversion={s.Power: {s.Wind: -1}},
    capex=1462000,
    fopex=43000,
    vopex=4953,
    capacity=1000,
    land_use=30,
    label='Wind mill array',
    basis='MW',
)
s.PV = Process(
    sell_cost={s.Power: 30},
    credit={s.Power: pandas.DataFrame({'a': [2]})},
    conversion={s.Power: {1: {s.Solar: -1.2}, 2: {s.Solar: -1.5}}},
    capex=1333262,
    fopex=22623,
    capacity=True,
    label='Solar PV',
    basis='MW',
)
s.SMRH = Process(
    land_use=50,
    conversion={
        s.H2: {
            s.Power: -1.11,
            s.CH4: -3.76,
            s.H2O: -23.7,
            s.CO2_Vent: 1.03,
            s.CO2: 9.332,
        }
    },
    capex=2520000,
    fopex=945000,
    vopex=51.5,
    capacity=1000,
    label='Steam methane reforming + CCUS',
)
s.NGCC = Process(
    purchase_cost={s.CH4: 4, s.H2O: 20},
    conversion={
        s.Power: {
            s.CH4: -0.108,
            s.H2O: -10,
            s.CO2_Vent: 0.297 * 0.05,
            s.CO2: 0.297 * 0.95,
        }
    },
    capex=2158928,
    fopex=53320,
    vopex=4090,
    capacity=[1, 100],
    label='NGCC + 95% CC',
)
s.SMR = Process(
    capex=2400,
    fopex=800,
    vopex=0.03,
    conversion={s.H2: {s.Power: -1.11, s.CH4: -
                       3.76, s.H2O: -23.7, s.CO2_Vent: 9.4979}},
    capacity=1000,
    label='Steam methane reforming',
)
s.H2FC = Process(
    purchase_cost={s.H2: 2},
    discharge={s.Power: 20},
    conversion={s.Power: {s.H2: -0.050}},
    capex=1.6 * 10**6,
    vopex=3.5,
    capacity=1000,
    label='hydrogen fuel cell',
)
s.DAC = Process(
    capex=0.02536,
    fopex=0.634,
    conversion={s.CO2_DAC: {s.Power: -0.193, s.H2O: -4.048}},
    capacity=1000,
    label='Direct air capture',
)
# s.PSH = Process(conversion = {s.Power: 0.6}, capex = 3924781, fopex= 17820, vopex = 512.5, store = 10000, capacity=1000, label='Pumped storage hydropower', basis = 'MW')
s.ASMR = Process(
    consume={s.Uranium: 40},
    conversion={s.Power: {s.Uranium: -4.17 * 10 ** (-5), s.H2O: -3.364}},
    capex=7988951,
    fopex=0.04 * 0.730,
    capacity=1000,
    label='Small modular reactors (SMRs)',
)
s.AWE = Process(
    incidental=20,
    land_use=(0, 20),
    conversion={s.H2: {s.Power: -1, s.O2: 0.7632, s.H2O: -0.1753}},
    capex=1.1 * 10**6,
    fopex=16918,
    capacity=BigM,
    label='Alkaline water electrolysis (AWE)',
    citation='Demirhan et al. 2018 AIChE paper',
)

In [ ]:
a = ['a']
list(attrgetter('resources', 'processes')(s))

In [ ]:
[['a'], *list(attrgetter('resources', 'processes')(s))]

In [ ]:
from operator import attrgetter

In [ ]:
attrgetter('label')(s.Power)

In [ ]:
import operator

help(operator)

In [ ]:
from operator import imod

imod(max(s.h.discretizations), 365)

In [ ]:
s.h.discretizations

In [ ]:
help(attrgetter)

In [ ]:
s.Power.cons()

In [ ]:
s.cons()

In [ ]:
s.WF.produce.cons()

In [ ]:
s.ASMR.produce.cons()

In [ ]:
s.ASMR.cons()

In [ ]:
for i in s.processes:
    i.produce.cons()

In [ ]:
s.Power.cons()

In [ ]:
s.WF.produce.cons()

In [ ]:
s.ASMR.produce.cons()

In [ ]:
{'': 'raa'}

In [ ]:
capex = {s.goa: {s.t1: 20, s.t1: {'*': 20}}}

In [ ]:
{s.goa: {s.t0: {[30, 50], {'': 0.5}}}}

In [ ]:
s.cons

In [ ]:
s.WF.produce.cons()

In [ ]:
for i in s.resources:
    if hasattr(i, 'consume') and getattr(i, 'consume') is not None:
        i.consume.cons()

In [ ]:
s.processes

In [ ]:
s.Power.produce.cons()

In [ ]:
s.ASMR.produce.cons()

In [ ]:
for i in s.processes:
    print(i)
    i.produce.cons()
    print('___________')

In [ ]:
s.WF.produce.cons()

In [ ]:
s.WF.cons()

In [ ]:
type(s.WF.produce)

In [ ]:
s.WF.produce.component

In [ ]:
s.Power.cons()

In [ ]:
s.cons()

In [ ]:
s.H2.discharge.cons()

In [ ]:
s.SMR.discharge

In [ ]:
s.cons()

In [ ]:
s.AWE.produce

In [ ]:
s.H2.parameters

In [ ]:
c

In [ ]:
s.H2.cons()

In [ ]:
s.AWE.cons()

In [ ]:
s.AWE.consume

In [ ]:
s.H2.produce.cons()

In [ ]:
s.AWE.produce

In [ ]:
s.AWE.cons()

In [ ]:
s.H2.produce.cons()

In [ ]:
s.cons()

In [ ]:
Process.aspects()

In [ ]:
s.Power.consume.cons()

In [ ]:
s.Power.discharge.cons()

In [ ]:
s.Power.discharge.__dict__

In [ ]:
s.SMR.discharge

In [ ]:
s.WF.produce.__dict__

In [ ]:
Process.aspects()

In [ ]:
Process.resource_aspects()

In [ ]:
s.WF.discharge

In [ ]:
s.WF.produce

In [ ]:
s.WF.consume

In [ ]:
s.Wind.consume.__dict__

In [ ]:
s.WF.consume

In [ ]:
s.WF.produce.__dict__

In [ ]:
Process.__dataclass_fields__

In [ ]:
s.ASMR.__dict__

In [ ]:
s.WF.conversion.__dict__

In [ ]:
s.PV.conversion.__dict__

In [ ]:
s.LiI.store_cost

In [ ]:
s.cons()

In [ ]:
s.LiI_d.store_loss

In [ ]:
s.LiI_d.store_cost

In [ ]:
s.LiI.conversion.conversion

In [ ]:
s.LiI_d.conversion.conversion

In [ ]:
s.processes

In [ ]:
s.Power_in_LiI.cons()

In [ ]:
s.Power.cons()

In [ ]:
s.LiI_d.__dict__

In [ ]:
s.LiI_d.cons()

In [ ]:
s.LiI.cons()

In [ ]:
s.Power_in_LiI.__dict__

In [ ]:
s.processes

In [ ]:
s.resources

In [ ]:
s.processes

In [ ]:
s.Power.cons()

In [ ]:
s.Power.cons()

In [ ]:
s.LiI.cons()

In [ ]:
s.PV.cons()

In [ ]:
s.cons()

In [ ]:
len(set(s.constraints))

In [ ]:
a = {s.Power: {1: {s.Solar: -1.2}, 2: {s.Solar: -1.5}}}

In [ ]:
b = {1: {s.Solar: -1.2}, 2: {s.Solar: -1.5}}

In [ ]:
[k for k, l in j.items() for i, j in b.items()]

In [ ]:
WF.capacity.params[0].ub
WF.capacity.params[0].lb

In [ ]:
s.WF.cons()

In [ ]:
s.WF.params()

In [ ]:
Power.consume_scale

In [ ]:
i = tuple(dict.fromkeys(['a', 'b', 'c', 'b']).keys())

In [ ]:
f'{i}'

In [ ]:
WF.eqns()

In [ ]:
WF.capacity.indices

In [ ]:
PV.eqns()

In [ ]:
H2.eqns()

In [ ]:
NGCC.purchase_cost.eqns()

In [ ]:
NGCC.eqns()

In [ ]:
PV.eqns()

In [ ]:
Power.eqns()

In [ ]:
Power.sell_cost.indices

In [ ]:
H2FC.eqns()

In [ ]:
H2FC.purchase_cost.eqns()

In [ ]:
WF.eqns()

In [ ]:
WF.produce.params[0]

In [ ]:
WF.store

In [ ]:
Power.eqns()

In [ ]:
WF.eqns()

In [ ]:
PV.eqns()

In [ ]:
SMRH.eqns()

In [ ]:
Solar.eqns()

In [ ]:
Power.eqns()

In [ ]:
AWE.eqns()

In [ ]:
Power.eqns()

In [ ]:
H2.eqns()

In [ ]:
Uranium.consume.eqns()

In [ ]:
Uranium.eqns()

In [ ]:
NGCC.eqns()

In [ ]:
process_list = {
    LiI,
    WF,
    PV,
    SMRH,
    SMR,
    DAC,
    AWE,
    H2_L_c,
    H2_L_d,
    AQoff_SMR,
    ASMR,
    PSH,
    NGCC,
}

## Declare locations

In [ ]:
cc = Factor(data=pandas.DataFrame(
    {'a': [2]}), scales=scales, apply_max_scaler=False)



houston = Location(




    name='HO',



    processes={LiI, PV, WF, SMRH, AWE, AQoff_SMR},




    credit={PV: 2},




    cap_max_localize={WF: 0.4, AWE: 23},




    credit_factor={PV: pandas.DataFrame({'a': [2]})},




    consume_localize={Solar: 2},



    demand={Power: (0, 1)},




    land_cost=10,
    land_cost_factor=cc,




    capacity_factor={PV: weather_ho[['dni']], WF: weather_ho[['wind_speed']]},





    demand_factor={Power: demand_ho},




    discharge_factor={Power: demand_ho},



    purchase_cost_factor={CH4: ng_price},




    land_max=True,
    scales=scales,




    label='Houston',




)




sandiego = Locatio
    scales=scales,







    processes={LiI, PV, WF, SMRH, AWE, AQoff_SMR},








    consume_localize={Solar: 3},




    land_cost=30,




    land_cost_factor=pandas.DataFrame({'a': [2]}),



    credit={SMRH: 30, AQoff_SMR: Theta((0, 23))},




    credit_factor={SMRH: pandas.DataFrame(
        {'a': [1 - 0.001 * i for i in range(365)]})},




    capacity_factor={PV: weather_sd[['dni']], WF: weather_sd[['wind_speed']]},




    demand_factor={Power: demand_sd},




    purchase_cost_factor={CH4: ng_price},




    discharge_localize={O2: 0.2},
    scales=scales,




    label='SanDiego',



)




newyork = Location(




    name='NY',




    processes={LiI, H2FC},



    land_max=(0, 50000),



    scales=scales,





    cap_max_localize={PV: 0.2, AWE: 34},




    capacity_factor={PV: weather_ny[['dni']], WF: weather_ny[['wind_speed']]},



    demand={Power: 40},



    demand_factor={Power: demand_ny},




    credit={PV: 7},




    purchase_cost_localize={CH4: 2},
    scales=scales,




    label='NewYork',




)






    scales=scales,







In [ ]:
PV.cap_max

In [ ]:
PV.factors

In [ ]:
houston.aspect

In [ ]:
houston.land_cost_factor

In [ ]:
# for i in houston.processes | sandiego.processes | newyork.processes:
# # for i in houston.resources | sandiego.resources | newyork.resources:
#     print(i, i.localizations)
#     print(i, i.factors)

In [ ]:
Power.ctype

## Declare transports

In [ ]:
# Train_H2 = Transport(name='Train_H2', resources={
#                      H2}, trans_max=bigM, trans_loss=0.03, trans_cost=1.667*10**(-3), label='Railway for hydrogen transportation')
# Grid = Transport(name='Grid', resources={
#                  Power}, trans_max=bigM, trans_loss=0.001, trans_cost=0.5*10**(-3), label='Railroad transport')


Train_H2 = Transport(
    name='Train_H2',
    resources={H2},
    cap_max=BigM,
    trans_loss=0.03,
    capex=1000,
    vopex=1.667,
    fopex=(0, 1),
    acid=34,
    eutf=55,
    label='Railway for hydrogen transportation',
)
Grid = Transport(
    name='Grid',
    resources={H2, Power},
    cap_max=BigM,
    trans_loss=0.001,
    capex=500,
    vopex=0.444,
    label='Railroad transport',
)

In [ ]:
Train_H2.cap_max

## Declare Network

In [ ]:
distance_matrix = [[2366], [2620]]

transport_matrix = [
    [[Train_H2]],
    [[Grid]],
]
capacity_factor = {
    (houston, newyork): {Train_H2: weather_sd[['wind_speed']]},
    (sandiego, newyork): {Train_H2: weather_sd[['wind_speed']]},
}

network = Network(
    name='network',
    scales=scales,
    land_max=(0, 5),
    land_max_factor=pandas.DataFrame({'a': [5]}),
    capacity_factor=capacity_factor,
    vopex_factor={(houston, sandiego): {Train_H2: weather_sd[['wind_speed']]}},
    sources=[houston, sandiego],
    sinks=[newyork],
    distance_matrix=distance_matrix,
    transport_matrix=transport_matrix,
)

In [ ]:
scenario = Scenario(
    name='scenario_full', network=network, scales=scales, label='full_case'
)

In [ ]:
scenario

In [ ]:
for i in houston.resources:
    print(i.ctype)

In [ ]:
houston.resources_produce

In [ ]:
ResourceType.all()

In [ ]:
ResourceType.resource_level()

In [ ]:
ResourceType.location_level()

In [ ]:
ResourceType.transport_level()

In [ ]:
ProcessType.all()

In [ ]:
ProcessType.process_level()

In [ ]:
ProcessType.location_level()

In [ ]:
LocationType.location_level()

In [ ]:
Power.transport

In [ ]:
Solar.availability

In [ ]:
Power.demand

In [ ]:
[((houston, sandiego), CO2), ((houston, sandiego), H2), ((houston, newyork), CO2)]

In [ ]:
comp_loc_dict = scenario.resources_locations_discharge

In [ ]:
scenario.locations_land

In [ ]:
scenario.locations_locations_land

In [ ]:
comp_loc_ordered_set = [(i, j)
                        for i in comp_loc_dict for j in comp_loc_dict[i]]

In [ ]:
scenario.resources_locations_store

In [ ]:
comp_loc_ordered_set

In [ ]:
comp_loc_ordered_set

In [ ]:
scenario.resources_locations_discharge

In [ ]:
scenario.resources_discharge

In [ ]:
scenario.processes_locations_expenditure

In [ ]:
resources_consume_locations = {
    resource: set() for resource in scenario.resources_consume
}
for resource in scenario.resources_consume:
    for location in scenario.locations:
        if hasattr(location, 'resources_consume'):
            if resource in location.resources_consume:
                resources_consume_locations[resource].add(location)

In [ ]:
resources_consume_locations = {
    resource: {
        location
        for location in scenario.locations
        if hasattr(location, 'resources_consume')
        and resource in location.resources_consume
    }
    for resource in scenario.resources_consume
}

In [ ]:
scenario.resources_consume

In [ ]:
resources_consume_locations = {
    resource: {
        location
        for location in self.locations
        if hasattr(location, 'resources_consume')
        and resource in location.resources_consume
    }
    for resource in self.resources_consume
}

In [ ]:
resources_consume_locations

In [ ]:
resources_demand_locations = {
    resource: {
        location
        for location in scenario.locations
        if hasattr(location, 'resources_demand')
        and resource in location.resources_demand
    }
    for resource in scenario.resources_demand
}

In [ ]:
scenario.resource_classifications()

In [ ]:
{
    component: {
        location
        for location in getattr(self, 'locations')
        if hasattr(location, f'{component_set}_{parameter.lower()}')
        and resource in getattr(location, f'{component_set}_{parameter.lower()}')
    }
    for resource in getattr(self, f'{component_set}_{parameter.lower()}')
}

In [ ]:
scenario.process_classifications()

In [ ]:
[
    ((houston.name, sandiego.name), CO2.name),
    ((houston.name, sandiego.name), H2.name),
    ((houston.name, newyork.name), CO2.name),
]

In [ ]:
help(ResourceParamType)

In [ ]:
from pyomo.environ import *

# from energiapy.utils.scale_utils import scale_pyomo_set

In [ ]:
scales1 = TemporalScale(discretization_list=[
                        1, 4], design_scale=0, scheduling_scale=1)

In [ ]:
type(scales1)

In [ ]:
scales2 = TemporalScale(
    discretization_list=[2, 2, 2], design_scale=0, scheduling_scale=2
)

In [ ]:
scales1.index_list

In [ ]:
scales2.index_list

In [ ]:
m = ConcreteModel()
m.scales = Set(scales1.list, initialize=scales1.scale)
m.scales_scheduling = scale_pyomo_set(
    instance=m, scale_level=scales1.scheduling_scale)
m.scales_design = scale_pyomo_set(instance=m, scale_level=scales1.design_scale)

In [ ]:
scales.scheduling_scale

In [ ]:
# m.a = Set(initialize = [(i, j) for i in resources_consume_locations for j in resources_consume_locations[i]])
# m.b = Set(initialize = [(i.name, j.name) for i in scenario.resources_locations_consume for j in scenario.resources_locations_consume[i]])
# m.c = Set(initialize = [((houston.name, sandiego.name), CO2.name), ((houston.name, sandiego.name), H2.name), ((houston.name, newyork.name), CO2.name)])
# m.A = Var(m.a, m.scales_scheduling, within = NonNegativeReals)
# m.B = Var(m.b, m.scales_design, within = NonNegativeReals)
# m.C = Var(m.c, m.scales_scheduling, within = NonNegativeReals)

In [ ]:
from pyomo.environ import *

m = ConcreteModel()

In [ ]:
m.d = Set(
    initialize=[
        (Train_H2.name, (houston.name, sandiego.name)),
        (Grid.name, (houston.name, sandiego.name)),
        (Grid.name, (houston.name, newyork.name)),
    ]
)
m.D = Var(m.d, m.scales_design, within=NonNegativeReals)

In [ ]:
m.f = Set(initialize=[Train_H2.name, Grid.name])
m.F = Var(m.f, m.scales_design, within=NonNegativeReals)

In [ ]:
m.g = Set(
    initialize=[
        (CO2.name, (houston.name, sandiego.name)),
        (Power.name, (houston.name, sandiego.name)),
        (H2.name, (houston.name, newyork.name)),
    ]
)

In [ ]:
m.G = Var(m.g, m.scales_scheduling, within=NonNegativeReals)

In [ ]:
m.G.pprint()

In [ ]:
m = ConcreteModel()

In [ ]:
import gurobipy

m = gurobipy.read('example.lp')

In [ ]:
import pandas

In [ ]:
a = pandas.DataFrame(data={'a': [1, 2, 3]})

In [ ]:
b = 

In [ ]:
class LPFileCreator:
    def __init__(self, objective, constraints, bounds, file_name="problem.lp"):
        self.objective = objective
        self.constraints = constraints
        self.bounds = bounds
        self.file_name = file_name

    def create_lp_file(self):
        content = self._generate_lp_content()
        with open(self.file_name, "w") as file:
            file.write(content)

    def _generate_lp_content(self):
        lp_content = []

        # Add Objective Function
        lp_content.append("\\* Problem Statement *\\")
        lp_content.append("\nMaximize")
        lp_content.append(f" obj: {self.objective}\n")

        # Add Constraints
        lp_content.append("Subject To")
        for i, constraint in enumerate(self.constraints, start=1):
            lp_content.append(f" c{i}: {constraint}")
        lp_content.append("")

        # Add Bounds
        lp_content.append("Bounds")
        for bound in self.bounds:
            lp_content.append(f" {bound}")
        lp_content.append("")

        # Add End
        lp_content.append("End")

        return "\n".join(lp_content)

In [ ]:
a = [7, 2, 3].sort()

In [ ]:
a

In [ ]:
sorted(a)

In [ ]:
houston.resources_consume

In [ ]:
[f'cons_{i.name}_' for i in houston.resources_consume]

In [ ]:
def tuples_to_strings(tuple_list):
    return ['_'.join(map(str, t)) for t in tuple_list]

In [ ]:
scales = TemporalScale([1, 4], design_scale=0, scheduling_scale=1)

In [ ]:
var = ['cons']  # , 'sell']

In [ ]:
res_list = ['res1', 'res2', 'res3']

In [ ]:
time_index = tuples_to_strings(scales.index_list[1])

In [ ]:
value = {'res1': 5, 'res2': 7, 'res3': 8}

In [ ]:
cons = list()

In [ ]:
for i, j, k in product(var, res_list, time_index):
    cons.append(f'{i}_{j}_{k} <= {value[j]}')

In [ ]:
ooo = {'res1': 7, 'res2': 5, 'res3': 9}

In [ ]:
k = [f'{ooo[i]} {i}' for i in res_list]

In [ ]:
obj = ' + '.join(k)
obj

In [ ]:
for i,j,k in product(var, res_list, time_index):


In [ ]:
bou = [f'0 <= {i}' for i in var]

In [ ]:
# Define the problem components
objective = "x + 2 y"
constraints = ["x + y >= 1", "x <= 2", "y <= 2"]
bounds = ["0 <= x", "0 <= y"]

# Create the LP file
# creator = LPFileCreator(objective, constraints, bounds, file_name="example.lp")

creator = LPFileCreator(obj, cons, bou, file_name="example.lp")
creator.create_lp_file()
print("LP file 'example.lp' created successfully.")

In [ ]:
import gurobipy as gb

a = gb.read('example.lp')

In [ ]:
a.optimize()

In [ ]:
Maximize
  x + y + z
Subject To
  c0: x + y = 1
  c1: x + 5 y + 2 z <= 10
  qc0: x + y + [ x ^ 2 - 2 x * y + 3 y ^ 2 ] <= 5
Bounds
  0 <= x <= 5
  z >= 2
Generals
  x y z
End


In [ ]:
H2O.purchase_cost

In [ ]:
H2O.cons_max

In [ ]:
m.optimize()

In [ ]:
m.getVars()

In [ ]:
plot_scenario.capacity_factor(
    scenario=scenario, location=houston, process=PV, fig_size=(9, 5), color='orange'
)

![cap fac pv](plots / cf_ho.png)

In [ ]:
plot_scenario.capacity_factor(
    scenario=scenario, location=sandiego, process=WF, fig_size=(9, 5), color='blue'
)

![cap fac wf](plots / wf_sd.png)

In [ ]:
plot_scenario.demand_factor(
    scenario=scenario, location=newyork, resource=Power, fig_size=(9, 5), color='red'
)

![df ny](plots / df_ny.png)

## Optimize for minimum cost

In [ ]:
milp_cost = formulate(
    scenario=scenario,
    constraints={
        Constraints.COST,
        Constraints.INVENTORY,
        Constraints.PRODUCTION,
        Constraints.RESOURCE_BALANCE,
        Constraints.TRANSPORT,
        Constraints.MODE,
        Constraints.NETWORK,
    },
    objective=Objective.COST,
)

**Ensure no discharge of power in Houston and San Diego**

While the demand for power is zero in these locations, Power is still a dischargeable resource. Hence, the discharge bounds need to be set to zero.
If not set to zero, the system chooses to dispense power instead of establishing storage networks.

In [ ]:
milp_cost.constraint_specific_location_discharge_ho = (
    constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location=houston
    )
)
milp_cost.constraint_specific_location_discharge_sd = (
    constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location=sandiego
    )
)

In [ ]:
results_cost = solve(
    scenario=scenario,
    instance=milp_cost,
    solver='gurobi',
    name="results_cost",
    print_solversteps=True,
)

**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(
    results=results_cost, source='SD', sink='NY', resource='H2', transport='Train_H2'
)

![sch t sdny](plots / sch_sdny.png)

In [ ]:
plot_results.transport(
    results=results_cost, source='HO', sink='NY', resource='H2', transport='Train_H2'
)

![sch hony](plots / sch_hony.png)

**Cost contribution by technology**

In [ ]:
plot_results.cost(
    results=results_cost,
    x=CostX.PROCESS_WISE,
    y=CostY.CAPEX,
    location='HO',
    fig_size=(8, 6),
)
plot_results.cost(
    results=results_cost,
    x=CostX.PROCESS_WISE,
    y=CostY.VOPEX,
    location='HO',
    fig_size=(8, 6),
)
plot_results.cost(
    results=results_cost,
    x=CostX.PROCESS_WISE,
    y=CostY.FOPEX,
    location='HO',
    fig_size=(8, 6),
)
plot_results.cost(
    results=results_cost,
    x=CostX.PROCESS_WISE,
    y=CostY.TOTAL,
    location='HO',
    fig_size=(8, 6),
)

![total cost ho](plots / total_ho.png)

**Location-wise cost contribution by technology**

In [ ]:
plot_results.cost(
    results=results_cost, x=CostX.LOCATION_WISE, y=CostY.CAPEX, fig_size=(8, 6)
)
plot_results.cost(
    results=results_cost, x=CostX.LOCATION_WISE, y=CostY.VOPEX, fig_size=(8, 6)
)
plot_results.cost(
    results=results_cost, x=CostX.LOCATION_WISE, y=CostY.FOPEX, fig_size=(8, 6)
)
plot_results.cost(
    results=results_cost, x=CostX.LOCATION_WISE, y=CostY.TOTAL, fig_size=(8, 6)
)

![total lw](plots / total_lw.png)

**Inventory and production schedules**

In [ ]:
plot_results.schedule(
    results=results_cost,
    y_axis='Inv',
    component='LiI_Power_stored',
    location='SD',
    fig_size=(9, 5),
    color='steelblue',
)

![inv lii](plots / lii_inv.png)

In [ ]:
plot_results.schedule(
    results=results_cost,
    y_axis='P',
    component='PV',
    location='SD',
    fig_size=(9, 5),
    color='steelblue',
)

![pv p](plots / pv_p.png)

## Optimize to maximize resource discharge 

In [ ]:
milp_demand = formulate(
    scenario=scenario,
    constraints={
        Constraints.COST,
        Constraints.INVENTORY,
        Constraints.PRODUCTION,
        Constraints.RESOURCE_BALANCE,
        Constraints.TRANSPORT,
        Constraints.MODE,
        Constraints.NETWORK,
    },
    objective=Objective.MAX_DISCHARGE,
    objective_resource=Power,
)

In [ ]:
milp_demand.constraint_specific_location_discharge_ho = (
    constraint_specific_location_discharge(
        instance=milp_demand, network_scale_level=0, bounds={Power: 0}, location=houston
    )
)
milp_demand.constraint_specific_location_discharge_sd = (
    constraint_specific_location_discharge(
        instance=milp_demand,
        network_scale_level=0,
        bounds={Power: 0},
        location=sandiego,
    )
)

In [ ]:
results_demand = solve(
    scenario=scenario,
    instance=milp_demand,
    solver='gurobi',
    name="results_demand",
    print_solversteps=True,
)

**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(
    results=results_demand, source='SD', sink='NY', resource='H2', transport='Train_H2'
)

![sch t sdny2](plots / sch_sdny2.png)

In [ ]:
plot_results.transport(
    results=results_demand, source='HO', sink='NY', resource='H2', transport='Train_H2'
)

![sch t hony](plots / sch_hony2.png)